In [24]:
###################################
#                                 
#             SETTINGS            
#                                 
###################################

# clearing the memory
rm(list = ls())

# installing pacman
if (require(pacman) == F) install.packages('pacman')
library(pacman)

# libraries
p_load(stargazer, ecr)
p_load(qcc)

# working directory
cd <- 'C:/Users/PC/Fair_Credit_Scoring-main/codes'
setwd(dirname(cd))


also installing the dependencies 'htmltools', 'htmlwidgets', 'rlang', 'viridis', 'plot3Drgl', 'plotly'


Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  无法打开URL'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"



  There are binary versions available but the source versions are later:
             binary source needs_compilation
htmltools   0.5.1.1  0.5.5              TRUE
htmlwidgets   1.5.3  1.6.2             FALSE
rlang        0.4.11  1.1.0              TRUE
viridis       0.6.1  0.6.2             FALSE
plot3Drgl     1.0.1  1.0.4             FALSE
plotly        4.9.3 4.10.1             FALSE
ecr           2.1.0  2.1.1              TRUE

  Binaries will be installed
package 'htmltools' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'htmltools'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"拷贝D:\ProgramData\Anaconda3\envs\fairness\Lib\R\library\00LOCK\htmltools\libs\x64\htmltools.dll到D:\ProgramData\Anaconda3\envs\fairness\Lib\R\library\htmltools\libs\x64\htmltools.dll时出了问题：Permission denied "
Warning message:
"restored 'htmltools'"


package 'rlang' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'rlang'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"拷贝D:\ProgramData\Anaconda3\envs\fairness\Lib\R\library\00LOCK\rlang\libs\x64\rlang.dll到D:\ProgramData\Anaconda3\envs\fairness\Lib\R\library\rlang\libs\x64\rlang.dll时出了问题：Permission denied "
Warning message:
"restored 'rlang'"


package 'ecr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PC\AppData\Local\Temp\RtmpqQ3Lvp\downloaded_packages


installing the source packages 'htmlwidgets', 'viridis', 'plot3Drgl', 'plotly'


Warning message in utils::install.packages(package, ...):
"installation of package 'htmlwidgets' had non-zero exit status"
Warning message in utils::install.packages(package, ...):
"installation of package 'viridis' had non-zero exit status"
Warning message in utils::install.packages(package, ...):
"installation of package 'plot3Drgl' had non-zero exit status"
Warning message in utils::install.packages(package, ...):
"installation of package 'plotly' had non-zero exit status"

ecr installed

Warning message:
"package 'ecr' was built under R version 3.6.3"
Warning message:
"package 'smoof' was built under R version 3.6.3"
Warning message in p_load(stargazer, ecr):
"Failed to install/load:
ecr"
Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6:
  无法打开URL'http://www.stats.ox.ac.uk/pub/RWin/bin/windows/contrib/3.6/PACKAGES'"


package 'qcc' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\PC\AppData\Local\Temp\RtmpqQ3Lvp\downloaded_packages



qcc installed

Warning message:
"package 'qcc' was built under R version 3.6.3"


In [26]:
###################################
#                                 
#           PARAMETERS            
#                                 
###################################

# paths
source(file.path(cd, 'code_00_working_paths.R'))

# names
datasets <- c('german',
              'bene',
              'taiwan',
              'uk',
              'pkdd',
              'gmsc',
              'homecredit')
metrics <- c('AUC',
             'acceptedLoans',
             'profitPerEUR',
             'statParityDiff', 
             'averageOddsDiff', 
             'predParityDiff')
models <- c('glm',
            'rf',
            'xgbTree',
            'nnet')
methods <- c('pre_reweighing',
             'pre_di_remover',
             'in_prej_remover',
             'in_adv_debias',
             'in_meta_alg',
             'post_roc',
             'post_eop',
             'post_platt',
             'bench_maxprof')

# params
num_data    <- length(datasets)
num_metrics <- length(metrics)
num_methods <- length(methods)
num_folds   <- 5

In [27]:
##################################
#                                
#         IMPORT RESULTS      
#                                
##################################

# model names
source(file.path(func_path, '98_param_grids.R'))

# placeholders
results <- array(NA, dim = c(num_data, num_folds, length(models), num_methods, num_metrics))

# extract results
for (data in datasets) {
  
  # feedback
  print(paste0('- ', data, '...'))
  
  # loop through folds and base models
  for (fold in seq(0, num_folds - 1)) {
    for (model in models) {
  
      # pre-processors
      results[which(datasets == data), fold + 1, which(models == model), 1, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold, '_RW_results.csv')), row.names = 1)[metrics, model]
      results[which(datasets == data), fold + 1, which(models == model), 2, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold, '_DI_results.csv')), row.names = 1)[metrics, model]
      
      # in-processors
      results[which(datasets == data), fold + 1, which(models == model), 3, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold,'_PR_results.csv')), row.names = 1)[metrics, 1]
      results[which(datasets == data), fold + 1, which(models == model), 4, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold,'_AD_results.csv')), row.names = 1)[metrics, 1]
      results[which(datasets == data), fold + 1, which(models == model), 5, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold,'_MA_results.csv')), row.names = 1)[metrics, 1]
      
      # post-processors
      results[which(datasets == data), fold + 1, which(models == model), 6, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold, '_ROC_results.csv')), row.names = 1)[metrics, model]
      results[which(datasets == data), fold + 1, which(models == model), 7, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold, '_EOP_results.csv')), row.names = 1)[metrics, model]
      results[which(datasets == data), fold + 1, which(models == model), 8, ] <-
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold, '_PL_results.csv')), row.names = 1)[metrics, model]
      
      # benchmarks
      results[which(datasets == data), fold + 1, which(models == model), 9, ] <- 
        read.csv(file.path(res_path, 'final', paste0(data, '_', fold, '_bench_maxprof_results.csv')), row.names = 1)[metrics, model]
    }
  }
}

# save results
saveRDS(results, file.path(res_path, 'final', 'aggregated_results.RDS'))

[1] "- german..."
[1] "- bene..."
[1] "- taiwan..."
[1] "- uk..."
[1] "- pkdd..."
[1] "- gmsc..."
[1] "- homecredit..."


In [4]:
##################################
#                                
#    PERFORMANCE PER DATA SET   
#                                
##################################

# load results
results <- readRDS(file.path(res_path, 'final', 'aggregated_results.RDS'))

# displayed metrics
rel_metrics <- c('AUC', 
                 'profitPerEUR',
                 'acceptedLoans',
                 'statParityDiff', 
                 'averageOddsDiff', 
                 'predParityDiff')

# extract results
for (data in datasets) {

  # placeholders
  tmp_results_me           <- matrix(nrow = num_methods, ncol = length(rel_metrics))
  rownames(tmp_results_me) <- methods
  colnames(tmp_results_me) <- rel_metrics
  tmp_results_se           <- tmp_results_me
  
  # extract results
  for (metric in rel_metrics) {
    vals <- NULL
    for (model in models) {
        tmp_res <- data.frame(results[which(datasets == data), , which(models == model), , which(metrics == metric)])
        vals    <- rbind(vals, tmp_res)
    }
    colnames(vals) <- methods
    tmp_results_me[ , which(rel_metrics == metric)] <- apply(vals, 2, function(x) mean(x, na.rm = T))
    tmp_results_se[ , which(rel_metrics == metric)] <- apply(vals, 2, function(x) sd(x / sqrt(length(x)), na.rm = T))
  }
  
  # prepare a table
  stargazer(tmp_results_me, type = 'text', title = data, digits = 4, 
            out = file.path(out_path, paste0('results_', data, '.html')))
}



##################################
#                                
#    AGGREGATED PERFORMANCE
#                                
##################################

# load results
results <- readRDS(file.path(res_path, 'final', 'aggregated_results.RDS'))

# displayed metrics
rel_metrics <- c('AUC', 
                 'profitPerEUR',
                 'statParityDiff', 
                 'averageOddsDiff', 
                 'predParityDiff')

# placeholders
tmp_results           <- matrix(nrow = num_methods, ncol = length(rel_metrics))
rownames(tmp_results) <- methods
colnames(tmp_results) <- rel_metrics

# extract results
for (metric in rel_metrics) {
  vals <- NULL
  for (model in models) {
    for (data in datasets) {
      tmp_res  <- data.frame(results[which(datasets == data), , which(models == model), , which(metrics == metric)])
      tmp_res  <- na.omit(tmp_res)
      vals     <- rbind(vals, tmp_res)
    }
  }
  colnames(vals) <- methods
  gaps  <- (vals - vals[, 'bench_maxprof']) / vals[, 'bench_maxprof'] * 100
  gaps  <- gaps[is.finite(rowSums(gaps)), ]
  tmp_results[ , which(rel_metrics == metric)] <- apply(gaps, 2, function(x) mean(x, na.rm = T))
}

# invert signs for fairness
tmp_results[, c('statParityDiff', 'averageOddsDiff', 'predParityDiff')] <- 
  -tmp_results[, c('statParityDiff', 'averageOddsDiff', 'predParityDiff')]

# add mean gains
mean_gains  <- apply(tmp_results[1:nrow(tmp_results)-1, ], 2, function(x) mean(x, na.rm = T))
tmp_results <- rbind(tmp_results[1:nrow(tmp_results)-1, ], mean_gains)

# export table
stargazer(tmp_results, title = 'Performance Gains)', type = 'text',  digits = 2, 
          out = file.path(out_path, 'performance_gains.html'))


german
                 AUC   profitPerEUR acceptedLoans statParityDiff averageOddsDiff predParityDiff
-----------------------------------------------------------------------------------------------
pre_reweighing  0.7604    0.0252       0.6113         0.2204         0.1752          0.1563    
pre_di_remover  0.8121    0.0494       0.6172         0.2989         0.1919          0.1249    
in_prej_remover 0.7933    0.0463       0.6112         0.3200         0.2091          0.1655    
in_adv_debias   0.7965    0.0502       0.6103         0.2528         0.1898          0.1705    
in_meta_alg     0.8074    0.0467       0.6158         0.2262         0.1117          0.1555    
post_roc        0.7124    0.0254       0.5985         0.1105         0.0881          0.2121    
post_eop        0.6999    0.0300       0.5965         0.0836         0.1475          0.2514    
post_platt      0.8012    0.0464       0.6139         0.4195         0.3369          0.1532    
bench_maxprof   0.8124    0.0492


Performance Gains)
                 AUC   profitPerEUR statParityDiff averageOddsDiff predParityDiff
---------------------------------------------------------------------------------
pre_reweighing  -3.52     -21.64        66.30           62.07          -22.23    
pre_di_remover   0.20     -10.63         6.59           5.96           -23.28    
in_prej_remover -0.32     -5.37         14.77           12.19         -208.33    
in_adv_debias   -6.23     -35.32        31.35           29.06         -173.28    
in_meta_alg     -3.62     -7.12         -8.17          -21.37         -103.97    
post_roc        -13.02    -42.59        78.61           76.63         -262.65    
post_eop        -16.02    -48.30        28.04           -9.21         -390.64    
post_platt      -0.51     -13.33        -82.65         -105.38         -81.43    
mean_gains      -5.38     -23.04        16.85           6.24          -158.23    
------------------------------------------------------------------------------

In [5]:
##################################
#                                
#       CORRELATION ANALYSIS      
#                                
##################################

# load results
results <- readRDS(file.path(res_path, 'final', 'aggregated_results.RDS'))

# displayed metrics
rel_metrics <- c('AUC', 
                 'profitPerEUR',
                 'statParityDiff', 
                 'averageOddsDiff', 
                 'predParityDiff')

# placeholders
tmp_results_me           <- matrix(nrow = num_methods, ncol = num_metrics)
rownames(tmp_results_me) <- methods
colnames(tmp_results_me) <- metrics

# extract results
for (data in datasets) {
  res <- list()
  for (metric in rel_metrics) {
    vals <- NULL
    for (model in models) {
        tmp_res <- data.frame(results[which(datasets == data), , which(models == model), , which(metrics == metric)])
        vals    <- rbind(vals, apply(tmp_res, 2, function(x) mean(x, na.rm = T)))
    }
    colnames(vals) <- methods
    res[[metric]]  <- vals
  }

  # transform data
  cor_methods <- methods[!(methods %in% 'base')]
  cor_vals    <- NULL
  res_tmp     <- list()
  for (method in cor_methods) {
    for (metric in metrics) {
      res_tmp[[metric]] <- res[[metric]][, method]
    }
    cor_vals <- rbind(cor_vals, data.frame(res_tmp))
  }
  
  # store dataset-specific correlation
  if (which(datasets == data) == 1) {
    cors <- cor(cor_vals, use = 'pairwise.complete.obs', method = 'spearman') / length(datasets)
  }else{
    cors <- cors + cor(cor_vals, use = 'pairwise.complete.obs', method = 'spearman') / length(datasets)
  }
}

# construct and export correlation matrix
colnames(cors) <- rel_metrics
rownames(cors) <- colnames(cors) 
cors[upper.tri(cors)] <- NA
stargazer(cors, type = 'text',  digits = 4, out = file.path(out_path, 'correlations.html'))


                  AUC   profitPerEUR statParityDiff averageOddsDiff predParityDiff
----------------------------------------------------------------------------------
AUC                1                                                              
profitPerEUR    0.7961       1                                                    
statParityDiff  0.4566     0.3647          1                                      
averageOddsDiff 0.3360     0.2898        0.9561            1                      
predParityDiff  -0.3537   -0.1643       -0.2125         -0.1342           1       
----------------------------------------------------------------------------------


In [28]:
##################################
#                                
#         PARETO FRONTIERS      
#                                
##################################

# helper function
source(file.path(func_path, '93_plot_pareto_frontier.R'))

# load results
results <- readRDS(file.path(res_path, 'final', 'aggregated_results.RDS'))

# relevant criterion
criterion <- 'averageOddsDiff'
if (criterion == 'statParityDiff')  {axt = 'Independence'}
if (criterion == 'averageOddsDiff') {axt = 'Separation'}
if (criterion == 'predParityDiff')  {axt = 'Sufficiency'}

# plot settings
par(mfrow = c(2,4), mar = c(4, 4, 2, 1) + 0.1)

# placeholder
per_02  <- NULL
per_min <- NULL
sep_min <- NULL

# loop through data sets
for (data in datasets) {
  
  # extract results
  data_vals <- NULL
  for (model in models) {
    for (method in methods) {
      tmp_res   <- data.frame(t(results[which(datasets == data), , which(models == model), which(methods == method), ]))
      data_vals <- rbind(data_vals, t(tmp_res))
    }
  }
  
  # column names
  colnames(data_vals) <- metrics  
  
  # preparation
  vals <- data_vals[, c(criterion, 'profitPerEUR')]
  vals <- na.omit(vals)
  vals <- as.matrix(vals, ncol = 2)
  vals <- vals[order(vals[, 1], decreasing = T), ]
  vals <- vals[!duplicated(vals), ]
  
  # drop non-dominated solutions
  vals[, 2] <- 1 - vals[, 2]
  vals      <- vals[which.nondominated(t(vals)), ]
  vals[, 2] <- 1 - vals[, 2]
  
  # compute profit percentage required to reduce fairness to 0.2
  per_02  <- c(per_02,  ((vals[1, 2] - vals[vals[, 1] <= 0.2, ][1, 2]) / vals[1, 2]))
  per_min <- c(per_min, ((vals[1, 2] - vals[nrow(vals), 2]) / vals[1, 2]))
  sep_min <- c(sep_min, vals[nrow(vals), 1])
  
  # plot settings for edge subplots
  par(mar = c(4,1.5,2,1.5) + 0.1)
  if (data %in% c('german', 'pkdd')) {par(mar = c(4, 3.5, 2, 1.5) + 0.1)}
  if (data %in% c('uk'))             {par(mar = c(4, 1.5, 2, 0.8) + 0.1)}
  
  # data id
  if (data == 'pkdd') {
    data.id <- 'pakdd'
  }else{
    data.id <- data
  }
  
  # plot Pareto front
  PlotParetoFront(vals, minimize = c(F, T), main = paste0('Data: ', data.id), objectives = c('', ''), cex = 2, pch = 21)
  mtext(cex = 0.85, side = 1, text = axt, line = 2.4)
  if (data %in% c('german', 'pkdd')) {
    mtext(cex = 0.80, side = 2, text = 'Profit per EUR Issued', line = 2.5)
  }
}

# export plot
dev.copy(pdf, file.path(out_path, paste0('pareto_frontiers.pdf')), width = 10, height = 5)
dev.off()

# display mean profit reductions
print(paste0('Profit reduction to reduce SEP to 0.2: ', round(100*mean(per_02), 2),  '%'))
print(paste0('Profit reduction to reduce SEP to min (', round(mean(sep_min), 2), '): ', round(100*mean(per_min), 2), '%'))


ERROR: Error in which.nondominated(t(vals)): 没有"which.nondominated"这个函数
